<a href="https://colab.research.google.com/github/toche7/PromptEngforDeveloper/blob/main/PromptLab1_GuidelinesforPrompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Guidelines for Prompting
ในบทนี้ เราจะได้ฝึกฝนหลักการสองประการในการใช้ prompt และเทคนิควิธีที่เกี่ยวข้องเพื่อเขียน  prompt ที่มีประสิทธิภาพสำหรับ  LLM (large language models).

```
# This is formatted as code
```



## Setup
#### Load the API key and relevant Python libaries.

ในกรณีที่เราใช้ Google Colab จำเป็นที่จะต้องติดตั้ง Library ของ OpenAI ก่อนตามตัวอย่างด้านล่าง และ
ใน Course นี้ผู้เรียนจะต้อง ไปเอา API Key มาจากเวปไซท์ของ OpenAI

In [1]:
#!pip -q install tiktoken
#!pip -q install cohere
!pip -q install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [2]:
from openai import OpenAI
import os
from google.colab import userdata
#os.environ["OPENAI_API_KEY"] = 'sk-xxx'
#client = OpenAI()
#os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('PROJ_OPENAI_KEY_MIRF')
client = OpenAI()

#### helper function
ตลอดการอบรมนี้ เราจะใช้ Model gpt-3.5-turbo

[chat completions endpoint](https://platform.openai.com/docs/guides/chat)

Function ด้านล่างจะช่วยให้เราเขียนโปรแกรมได้ง่ายขึ้น

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

## หลักการเขียน  Prompt
- **หลักการที่ 1: เขียนให้ชัดเจนและจำเพาะเจาะจง**
- **หลักการที่ 2: ให้เวลาในการคิด**




## Principle 1: เขียนให้ชัดเจนและจำเพาะเจาะจง

#### Tactic 1: ใช้สัญลักษณ์พิเศษในการแบ่งแยกข้อความ แต่ละส่วนให้ชัดเจน
- ทั้งนี้จะเป็นสัญลักษณ์อะไรก็ได้เช่น ```, """, < >, `<tag> </tag>`, `:`

In [4]:
text = f"""
You should express what you want a model to do by \
providing instructions that are as clear and \
specific as you can possibly make them. \
This will guide the model towards the desired output, \
and reduce the chances of receiving irrelevant \
or incorrect responses. Don't confuse writing a \
clear prompt with writing a short prompt. \
In many cases, longer prompts provide more clarity \
and context for the model, which can lead to \
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

It is important to provide clear and specific instructions to guide a model towards the desired output and reduce the chances of receiving irrelevant or incorrect responses, even if it means writing a longer prompt for more clarity and context.


#### Tactic 2:  กำหนดรูปแบบของ output ที่ต้องการ
- JSON, HTML

In [5]:
prompt = f"""
Generate a list of three made-up book titles along \
with their authors and genres.
Provide them in JSON  with the following keys:
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

[
    {
        "book_id": 1,
        "title": "The Midnight Garden",
        "author": "Elena Rivers",
        "genre": "Fantasy"
    },
    {
        "book_id": 2,
        "title": "Echoes of the Past",
        "author": "Nathan Black",
        "genre": "Mystery"
    },
    {
        "book_id": 3,
        "title": "Whispers in the Wind",
        "author": "Samantha Reed",
        "genre": "Romance"
    }
]


#### Tactic 3: ส่งให้ model ตรวจสอบเงื่อนไขว่าเป็นไปตามที่กำหนดหรือไม่

In [6]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \
water boiling. While that's happening, \
grab a cup and put a tea bag in it. Once the water is \
hot enough, just pour it over the tea bag. \
Let it sit for a bit so the tea can steep. After a \
few minutes, take out the tea bag. If you \
like, you can add some sugar or milk to taste. \
And that's it! You've got yourself a delicious \
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Pour the hot water over the tea bag.
Step 4 - Let the tea steep for a few minutes.
Step 5 - Remove the tea bag.
Step 6 - Add sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea.


In [7]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \
walk in the park. The flowers are blooming, and the \
trees are swaying gently in the breeze. People \
are out and about, enjoying the lovely weather. \
Some are having picnics, while others are playing \
games or simply relaxing on the grass. It's a \
perfect day to spend time outdoors and appreciate the \
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


#### Tactic 4: "Few-shot" prompting หมายถึงการทำให้ดูเป็นตัวอย่างที่เราต้องการ และให้ทำตาม

In [8]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \
valley flows from a modest spring; the \
grandest symphony originates from a single note; \
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: Resilience is like a tree that withstands the fiercest storms, bending but never breaking. It is the ability to bounce back from adversity, to find strength in the face of challenges, and to keep moving forward no matter what obstacles may come your way. Just like the tree, you too can weather the storms of life and emerge stronger on the other side.


## Principle 2: "ให้เวลาคิด"

#### Tactic 1: กำหนดขั้นตอนในการทำงานใดงานหนึ่งให้ชัดเจน อย่าคิดลัดนะ  

In [9]:
text = f"""
In a charming village, siblings Jack and Jill set out on \
a quest to fetch water from a hilltop \
well. As they climbed, singing joyfully, misfortune \
struck—Jack tripped on a stone and tumbled \
down the hill, with Jill following suit. \
Though slightly battered, the pair returned home to \
comforting embraces. Despite the mishap, \
their adventurous spirits remained undimmed, and they \
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into Thai.
3 - List each name in the Thai summary.
4 - Output a json object that contains the following \
keys: thai_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1 - Jack and Jill go on a quest to fetch water from a well, but encounter misfortune on the way back home.

2 - แจ็ค และ จิล ไปผจญภัยเพื่อเอาน้ำจากบ่อ แต่เกิดความโชคร้ายในการกลับบ้าน

3 - แจ็ค, จิล

4 - 
{
  "thai_summary": "แจ็ค และ จิล ไปผจญภัยเพื่อเอาน้ำจากบ่อ แต่เกิดความโชคร้ายในการกลับบ้าน",
  "num_names": 2
}


#### สั่งให้แสดงในรูปแบบที่ต้องการได้

In [ ]:
prompt_2 = f"""
Your task is to perform the following actions:
1 - Summarize the following text delimited by
  <> with 1 sentence.
2 - Translate the summary into Thai.
3 - List each name in the Thai summary.
4 - Output a json object that contains the
  following keys: thai_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: Jack and Jill went on a quest to fetch water but had a mishap on the way back home.

Translation: แจ็คและจิลไปผจญภัยเพื่อเอาน้ำ แต่มีอุบัติเหตุในขณะกลับบ้าน

Names: Jack, Jill

Output JSON: 
{
  "thai_summary": "แจ็คและจิลไปผจญภัยเพื่อเอาน้ำ แต่มีอุบัติเหตุในขณะกลับบ้าน",
  "num_names": 2
}


#### Tactic 2:  สั่งให้แบบจำลองแสดงวิธีการคิดหรือทำงานออกมาก่อนที่จะสรุปผล

In [10]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is correct. The total cost for the first year of operations as a function of the number of square feet is 450x + 100,000.


- เราจะพบว่าคำตอบที่ได้จะยังไม่ถูกต้อง
- เราสามารถแก้ไขได้โดยการสั้งให้ model ดำเนินการเป็นขั้นตอนก่อนที่จะตอบคำถามนั้น

In [11]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total.
- Then compare your solution to the student's solution \
and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
```
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

The actual solution is correct.

Actual solution:
Total cost = Land cost + Solar panel cost + Maintenance cost
Total cost = $100x + $250x + $100,000 + $10x
Total cost = $360x + $100,000

Is the student's solution the same as actual solution just calculated:
```
No
```
Student grade:
```
Incorrect
```


## Model Limitations: Hallucinations
- Let try about ITM.

In [20]:
prompt = f"""
Tell me about  ITM, Faculy of Engineering, Mahidol University
"""
response = get_completion(prompt)
print(response)

The International College of Mahidol University (ITM) is a leading international college in Thailand that offers a variety of undergraduate and graduate programs in engineering, business, and science. The Faculty of Engineering at ITM is known for its high-quality education and research in various engineering disciplines such as civil engineering, mechanical engineering, electrical engineering, and computer engineering.

The faculty is home to a diverse and talented group of faculty members who are experts in their respective fields and are dedicated to providing students with a well-rounded education that prepares them for successful careers in the engineering industry. The faculty also has state-of-the-art facilities and laboratories that allow students to gain hands-on experience and practical skills in their chosen field of study.

Overall, the Faculty of Engineering at ITM is committed to providing students with a world-class education that prepares them to become innovative and e

## Try experimenting on your own!

#### Ex 1. จงเขียน Prompt เพื่อตรวจข้อสอบต่อไปนี้ว่าถูกต้องหรือไม่
**question**: รูปสามเหลี่ยมรูปหนึ่งมีพื้นที่ 32 ตารางเซนติเมตร มีฐานยาว 8 เซนติเมตร รูปสามเหลี่ยมรูปนี้สูงกี่เซนติเมตร<br>
**answer** : 8  เซนติเมตร


In [26]:
question = 'รูปสามเหลี่ยมรูปหนึ่งมีพื้นที่ 32 ตารางเซนติเมตร มีฐานยาว 8 เซนติเมตร รูปสามเหลี่ยมรูปนี้สูงกี่เซนติเมตร'
answer = '8 เซนติเมตร'

In [27]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total.
- Then compare your solution to the student's solution \
and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
{question}
```
Student's solution:
```
{answer}
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

หากพื้นที่ของสามเหลี่ยมคำนวณได้จากสูตร Area = 0.5 * base * height
32 = 0.5 * 8 * height
32 = 4 * height
height = 32 / 4
height = 8

ดังนั้น ความสูงของสามเหลี่ยมคือ 8 เซนติเมตร

Is the student's solution the same as actual solution just calculated:
```
yes
```
Student grade:
```
correct
```


#### Ex 2.
จงเขียน Prompt เพื่อสรุปข่าวต่อไปนี้สำหรับใส่ลงไปใน Facebook

"เมื่อวันที่ 22 เมษายน 2567 สถาบันคุณวุฒิวิชาชีพ นำโดย คุณวรากร ธนูสนธิ์ หัวหน้ากลุ่มงานและคณะผู้ตรวจประเมิน ได้เข้าตรวจประเมิน “ศูนย์ฝึกอบรมฐานสมรรถนะตามมาตรฐานคุณวุฒิวิชาชีพ” ที่ดำเนินงานโดยกลุ่มสาขาวิชาเทคโนโลยีการจัดการระบบสารสนเทศ คณะวิศวกรรมศาสตร์ มหาวิทยาลัยมหิดล ตามคำขอการรับรองให้เป็นองค์กรที่ทำหน้าที่ให้บริการฝึกอบรมตามข้อกำหนดของสถาบันฯ โดยในงานตรวจประเมินครั้งนี้ มี ผศ.ดร. ทวีศักดิ์ สมานชื่น หัวหน้าโครงการศูนย์ฝึกฯ ได้ให้การต้อนรับและนำเสนอรายงานต่อคณะผู้ตรวจ พร้อมกับบุคลากรที่เกี่ยวข้อง อีกทั้งมีการตรวจสถานที่ที่ใช้ในการฝึกอบรม ตรวจความพร้อมเครื่องมือ และอุปกรณ์ที่เกี่ยวข้องในการฝึกอบรม จากการตรวจประเมิน สถาบันคุณวุฒิวิชาชีพได้สรุปผลการประเมินพร้อมข้อเสนอแนะเพิ่มเติมที่ให้องค์กรที่ทำหน้าที่ให้บริการฝึกอบรม การตรวจประเมินครั้งนี้ ถือเป็นการยกระดับมาตรฐานการฝึกอบรม หลักสูตร สาขาวิชาชีพอุตสาหกรรมดิจิทัล เป็นมาตรฐานองค์ฝึกอบรมตามข้อกำหนดของสถาบันฯ"



In [34]:
text = f"""
เมื่อวันที่ 22 เมษายน 2567 สถาบันคุณวุฒิวิชาชีพ นำโดย คุณวรากร ธนูสนธิ์ หัวหน้ากลุ่มงานและคณะผู้ตรวจประเมิน\
ได้เข้าตรวจประเมิน “ศูนย์ฝึกอบรมฐานสมรรถนะตามมาตรฐานคุณวุฒิวิชาชีพ” ที่ดำเนินงานโดยกลุ่มสาขาวิชาเทคโนโลยีการจัดการระบบสารสนเทศ\
คณะวิศวกรรมศาสตร์ มหาวิทยาลัยมหิดล ตามคำขอการรับรองให้เป็นองค์กรที่ทำหน้าที่ให้บริการฝึกอบรมตามข้อกำหนดของสถาบันฯ\
โดยในงานตรวจประเมินครั้งนี้ มี ผศ.ดร. ทวีศักดิ์ สมานชื่น หัวหน้าโครงการศูนย์ฝึกฯ ได้ให้การต้อนรับและนำเสนอรายงานต่อคณะผู้ตรวจ พร้อมกับบุคลากรที่เกี่ยวข้อง\
อีกทั้งมีการตรวจสถานที่ที่ใช้ในการฝึกอบรม ตรวจความพร้อมเครื่องมือ และอุปกรณ์ที่เกี่ยวข้องในการฝึกอบรม จากการตรวจประเมิน\
สถาบันคุณวุฒิวิชาชีพได้สรุปผลการประเมินพร้อมข้อเสนอแนะเพิ่มเติมที่ให้องค์กรที่ทำหน้าที่ให้บริการฝึกอบรม การตรวจประเมินครั้งนี้\
ถือเป็นการยกระดับมาตรฐานการฝึกอบรม หลักสูตร สาขาวิชาชีพอุตสาหกรรมดิจิทัล เป็นมาตรฐานองค์ฝึกอบรมตามข้อกำหนดของสถาบันฯ
"""
prompt = f"""
From the text delimited by triple backticks, perform the following task.
Create the Head News of the given text.
Write the text body as facebook fanpage style.
Write in Thai.

Use the following format.
Head News :
Body :
```{text}```
"""
response = get_completion(prompt)
print(response)

Head News : สถาบันคุณวุฒิวิชาชีพ ได้รับการตรวจประเมินและยกระดับมาตรฐานการฝึกอบรม

Body : 
สถาบันคุณวุฒิวิชาชีพ ได้รับการตรวจประเมิน "ศูนย์ฝึกอบรมฐานสมรรถนะตามมาตรฐานคุณวุฒิวิชาชีพ" โดยคณะผู้ตรวจประเมินจากสถาบันคุณวุฒิวิชาชีพ ซึ่งได้ให้ข้อเสนอแนะเพิ่มเติมให้องค์กรที่ทำหน้าที่ให้บริการฝึกอบรม การตรวจประเมินครั้งนี้ถือเป็นการยกระดับมาตรฐานการฝึกอบรม หลักสูตร สาขาวิชาชีพอุตสาหกรรมดิจิทัล เป็นมาตรฐานองค์ฝึกอบรมตามข้อกำหนดของสถาบันคุณวุฒิวิชาชีพ มากขึ้น นอกจากนี้ยังมีการตรวจสถานที่ที่ใช้ในการฝึกอบรม ตรวจความพร้อมเครื่องมือ และอุปกรณ์ที่เกี่ยวข้องในการฝึกอบรมอีกด้วยครับ/ค่ะ 🌟👏🏼🎉
#คุณวุฒิวิชาชีพ #ศูนย์ฝึกอบรม #การตรวจประเมิน #มาตรฐานการฝึกอบรม
